<a href="https://colab.research.google.com/github/wsyuei/wsyuei-stock-markets-analytics-zoomcamp-2024/blob/master/Homework%20Submission/Homework1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install main library YFinance
!pip install yfinance

In [2]:
# IMPORTS
import numpy as np
import pandas as pd

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import date

In [3]:
end = date.today()
print(f'Year = {end.year}; month= {end.month}; day={end.day}')

start = date(year=end.year-70, month=end.month, day=end.day)
print(f'Period for indexes: {start} to {end} ')

Year = 2024; month= 4; day=22
Period for indexes: 1954-04-22 to 2024-04-22 


# **Question 1: [Macro] Average growth of GDP in 2023**
What is the average growth (in %) of GDP in 2023?

Download the timeseries Real Gross Domestic Product (GDPC1) from FRED (https://fred.stlouisfed.org/series/GDPC1). Calculate year-over-year (YoY) growth rate (that is, divide current value to one 4 quarters ago). Find the average YoY growth in 2023 (average from 4 YoY numbers). Round to 1 digit after the decimal point: e.g. if you get 5.66% growth => you should answer 5.7

In [30]:
# Real Potential Gross Domestic Product (GDPPOT), Billions of Chained 2012 Dollars, QUARTERLY
# https://fred.stlouisfed.org/series/GDPC1
gdppot = pdr.DataReader("GDPC1", "fred", start='2022-01-01')

In [32]:
gdppot['gdpc1_us_yoy'] = gdppot.GDPC1/gdppot.GDPC1.shift(4)-1
gdppot['gdpc1_us_qoq'] = gdppot.GDPC1/gdppot.GDPC1.shift(1)-1
gdppot['gdpc1_us_yoy'].mean()

0.02540443392867625

# **Question 2. [Macro] Inverse "Treasury Yield"**
Find the min value of (dgs10-dgs2) after since year 2000 (2000-01-01) and write it down as an answer, round to 1 digit after the decimal point.

Download DGS2 and DGS10 interest rates series (https://fred.stlouisfed.org/series/DGS2, https://fred.stlouisfed.org/series/DGS10). Join them together to one dataframe on date (you might need to read about pandas.DataFrame.join()), calculate the difference dgs10-dgs2 daily.

(Additional: think about what does the "inverted yield curve" mean for the market and investors? do you see the same thing in your country/market of interest? Do you think it can be a good predictive feature for the models?)

In [20]:
# # "Core CPI index", MONTHLY
# https://fred.stlouisfed.org/series/CPILFESL
# The "Consumer Price Index for All Urban Consumers: All Items Less Food & Energy"
# is an aggregate of prices paid by urban consumers for a typical basket of goods, excluding food and energy.
# This measurement, known as "Core CPI," is widely used by economists because food and energy have very volatile prices.
dgs2 = pdr.DataReader("DGS2", "fred", start='2000-01-01')
dgs10 = pdr.DataReader("DGS10", "fred", start='2000-01-01')

In [22]:
dgs2_10 = pd.merge(dgs2, dgs10, on='DATE')
dgs2_10['result'] = dgs2_10['DGS10'] - dgs2_10['DGS2']
print(dgs2_10['result'].min())

-1.0800000000000005


# **Question 3. [Index] Which Index is better recently?**
Compare S&P 500 and IPC Mexico indexes by the 5 year growth and write down the largest value as an answer (%)

Download on Yahoo Finance two daily index prices for S&P 500 (^GSPC, https://finance.yahoo.com/quote/%5EGSPC/) and IPC Mexico (^MXX, https://finance.yahoo.com/quote/%5EMXX/). Compare 5Y growth for both (between 2019-04-09 and 2024-04-09). Select the higher growing index and write down the growth in % (closest integer %). E.g. if ratio end/start was 2.0925 (or growth of 109.25%), you need to write down 109 as your answer.

(Additional: think of other indexes and try to download stats and compare the growth? Do create 10Y and 20Y growth stats. What is an average yearly growth rate (CAGR) for each of the indexes you select?)

In [39]:
# Other indexes: https://stooq.com/t/

# SPX= S&P500
spx_index = yf.download(tickers ='^GSPC', start = '2019-04-09', end = '2024-04-09')
mxx_index = yf.download(tickers ='^MXX', start = '2019-04-09', end = '2024-04-09')
mxx_index.head()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-04-09,45346.828125,45346.828125,44854.531250,45151.628906,45151.628906,191575000
2019-04-10,45204.750000,45219.410156,44850.109375,44909.140625,44909.140625,145314900
2019-04-11,44872.531250,44966.500000,44373.488281,44580.058594,44580.058594,109090000
2019-04-12,44767.671875,44888.699219,44534.378906,44686.058594,44686.058594,143662400
2019-04-15,44649.738281,44900.929688,44347.531250,44625.781250,44625.781250,108627100


In [45]:
spx_index['adj_close_last_year'] = spx_index['Adj Close'].shift(252)
mxx_index['adj_close_last_year'] = mxx_index['Adj Close'].shift(252)

spx_index['yoy_growth'] = spx_index['Adj Close'] / spx_index['adj_close_last_year'] -1
mxx_index['yoy_growth'] = mxx_index['Adj Close'] / mxx_index['adj_close_last_year'] -1

In [53]:
print(spx_index['yoy_growth'].mean())
print(mxx_index['yoy_growth'].mean())

0.1250282080646889
0.07461070929886846


# **Question 4. [Stocks OHLCV] 52-weeks range ratio (2023) for the selected stocks**
Find the largest range ratio [=(max-min)/max] of Adj.Close prices in 2023

Download the 2023 daily OHLCV data on Yahoo Finance for top6 stocks on earnings (https://companiesmarketcap.com/most-profitable-companies/): 2222.SR,BRK-B, AAPL, MSFT, GOOG, JPM.

Here is the example data you should see in Pandas for "2222.SR": https://finance.yahoo.com/quote/2222.SR/history

Calculate maximum-minimim "Adj.Close" price for each stock and divide it by the maximum "Adj.Close" value. Round the result to two decimal places (e.g. 0.1575 will be 0.16)

(Additional: why this may be important for your research?)

In [67]:
# INDEXES from Yahoo Finance
# DAX index (XETRA - XETRA Delayed Price. Currency in EUR)
# WEB: https://finance.yahoo.com/quote/%5EGDAXI
sr_daily = yf.download(tickers = "2222.SR", start = '2023-01-01', end = '2023-12-31', period = "max", interval = "1d")
brkb_daily = yf.download(tickers = "BRK-B", start = '2023-01-01', end = '2023-12-31', period = "max", interval = "1d")
aapl_daily = yf.download(tickers = "AAPL", start = '2023-01-01', end = '2023-12-31', period = "max", interval = "1d")
msft_daily = yf.download(tickers = "MSFT", start = '2023-01-01', end = '2023-12-31', period = "max", interval = "1d")
goog_daily = yf.download(tickers = "GOOG", start = '2023-01-01', end = '2023-12-31', period = "max", interval = "1d")
jpm_daily = yf.download(tickers = "JPM", start = '2023-01-01', end = '2023-12-31', period = "max", interval = "1d")

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [70]:
print((sr_daily['Adj Close'].max() - sr_daily['Adj Close'].min())/ sr_daily['Adj Close'].max() )
print((brkb_daily['Adj Close'].max() - brkb_daily['Adj Close'].min())/ brkb_daily['Adj Close'].max() )
print((aapl_daily['Adj Close'].max() - aapl_daily['Adj Close'].min())/ aapl_daily['Adj Close'].max() )
print((msft_daily['Adj Close'].max() - msft_daily['Adj Close'].min())/ msft_daily['Adj Close'].max() )
print((goog_daily['Adj Close'].max() - goog_daily['Adj Close'].min())/ goog_daily['Adj Close'].max() )
print((jpm_daily['Adj Close'].max() - jpm_daily['Adj Close'].min())/ jpm_daily['Adj Close'].max() )

0.21393070882746015
0.20775750091289963
0.37244419224463476
0.4242066914981641
0.3924520921912013
0.2824993421323472


# **Question 5. [Stocks] Dividend Yield**
Find the largest dividend yield for the same set of stocks

Use the same list of companies (2222.SR,BRK-B, AAPL, MSFT, GOOG, JPM) and download all dividends paid in 2023. You can use get_actions() method or .dividends field in yfinance library (https://github.com/ranaroussi/yfinance?tab=readme-ov-file#quick-start)

Sum up all dividends paid in 2023 per company and divide each value by the closing price (Adj.Close) at the last trading day of the year.

Find the maximum value in % and round to 1 digit after the decimal point. (E.g., if you obtained $1.25 dividends paid and the end year stock price is $100, the dividend yield is 1.25% -- and your answer should be equal to 1.3)

In [90]:
#sr
sr = yf.Ticker('2222.SR')
sr_df = sr.get_actions()
sr_df = sr_df[(sr_df.index >='2023-01-01') & (sr_df.index <='2023-12-31')]
print(sr_df['Dividends'].sum()/sr_daily['Adj Close'].iloc[-1])

#brkb
brkb = yf.Ticker('BRK-B')
brkb_df = brkb.get_actions()
brkb_df = brkb_df[(brkb_df.index >='2023-01-01') & (brkb_df.index <='2023-12-31')]
print(brkb_df['Dividends'].sum()/brkb_daily['Adj Close'].iloc[-1])

#aapl
aapl = yf.Ticker('AAPL')
aapl_df = aapl.get_actions()
aapl_df = aapl_df[(aapl_df.index >='2023-01-01') & (aapl_df.index <='2023-12-31')]
print(aapl_df['Dividends'].sum()/aapl_daily['Adj Close'].iloc[-1])

#msft
msft = yf.Ticker('MSFT')
msft_df = msft.get_actions()
msft_df = msft_df[(msft_df.index >='2023-01-01') & (msft_df.index <='2023-12-31')]
print(msft_df['Dividends'].sum()/msft_daily['Adj Close'].iloc[-1])

#goog
goog = yf.Ticker('GOOG')
goog_df = goog.get_actions()
goog_df = goog_df[(goog_df.index >='2023-01-01') & (goog_df.index <='2023-12-31')]
print(goog_df['Dividends'].sum()/goog_daily['Adj Close'].iloc[-1])

#goog
jpm = yf.Ticker('JPM')
jpm_df = jpm.get_actions()
jpm_df = jpm_df[(jpm_df.index >='2023-01-01') & (jpm_df.index <='2023-12-31')]
print(jpm_df['Dividends'].sum()/jpm_daily['Adj Close'].iloc[-1])

0.027701498195032558
0.0
0.004940592304162832
0.007433143940964608
0.0
0.024096079718909574
